In [39]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import tensorflow_recommenders as tfrs

In [40]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

Let's pretend we have 3 users:
1. Clarissa is a vegetarian who is lactose intolerant. 🥝🍅
2. Martin is an athlete who prefers meat dishes to vegetable dishes. 🍖🥩
3. Stella - has no special preferences, eats mostly everything. 🍲🍠

In [3]:
clarissa = {'id':'clr', 'prefers': ['vegetables', 'vegan', 'nomeat', 'vegetarian'], 'hates': ['meat', 'lactose']}
martin = {'id':'mrt', 'prefers': 'meat', 'hates': 'vegetables'}
stella = {'id':'stl', 'prefers': '', 'hates': ''}

We will additionally enter the factors Calories, Protein, Fat, Carbohydrates for our requirements. **(The numbers were taken at random and do not reflect proportions or recommendations - the numbers are just an example)**
* Clarissa - 2000/100/60/120
* Martin, 2600/150/100/328.
* Stella - 2200/80/50/100

In [4]:
def set_requirements(user, calories, proteins, fats, carbs):
  user['calories'] = calories
  user['proteins'] = proteins
  user['fats'] = fats
  user['carbs'] = carbs
set_requirements(clarissa, 2000, 100, 60, 120)
set_requirements(martin, 2600, 150, 100, 328)
set_requirements(martin, 2200, 80, 50, 100)


# Data processing 📊

Loading recipes and user_interactions. Nutritions showed like:calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) , and carbohydrates (PDV)

In [41]:
raw_recipes = pd.read_csv('./sample_data/RAW_recipes.csv')
raw_interactions = pd.read_csv('./sample_data/RAW_interactions.csv')
raw_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


Merge data interactions with recipe infos

In [42]:
interactions_with_recipe_info = pd.merge(raw_interactions, raw_recipes[['name', 'tags', 'ingredients', 'id']], left_on='recipe_id', right_on='id', how='left')
interactions_with_recipe_info = interactions_with_recipe_info[['user_id', 'recipe_id', 'rating', 'name', 'tags', 'ingredients']]
interactions_with_recipe_info.rename(columns={'name': 'recipe_name'}, inplace=True)
interactions_with_recipe_info.head()

,user_id,recipe_id,rating,recipe_name,tags,ingredients
0,38094,40893,4,white bean green chile pepper soup,"['weeknight', 'time-to-make', 'course', 'main-...","['great northern beans', 'yellow onion', 'dice..."
1,1293707,40893,5,white bean green chile pepper soup,"['weeknight', 'time-to-make', 'course', 'main-...","['great northern beans', 'yellow onion', 'dice..."
2,8937,44394,4,devilicious cookie cake delights,"['30-minutes-or-less', 'time-to-make', 'course...","[""devil's food cake mix"", 'vegetable oil', 'eg..."
3,126440,85009,5,baked potato toppings,"['15-minutes-or-less', 'time-to-make', 'course...","['mayonnaise', 'salsa', 'cheddar cheese', 'ref..."
4,57222,85009,5,baked potato toppings,"['15-minutes-or-less', 'time-to-make', 'course...","['mayonnaise', 'salsa', 'cheddar cheese', 'ref..."


In [43]:
import ast

def convert_to_list(data_str):
  try:
    return ast.literal_eval(data_str)
  except (SyntaxError, ValueError):
    return []

In [44]:
interactions_with_recipe_info['ingredients_str'] = interactions_with_recipe_info['ingredients'].apply(convert_to_list)
interactions_with_recipe_info['tags_str'] = interactions_with_recipe_info['tags'].apply(convert_to_list)

raw_recipes['ingredients_str'] = raw_recipes['ingredients'].apply(convert_to_list)
raw_recipes['tags_str'] = raw_recipes['tags'].apply(convert_to_list)

In [12]:
interactions_with_recipe_info['ingredients_str'].str.len().unique()

array([ 9,  4, 13,  2, 12, 10, 16,  8, 14, 15,  7, 11,  6,  5, 23, 22, 17,
        3, 18, 19, 21, 26, 29, 24, 20, 27, 25, 33, 30, 31,  1, 28, 35, 32,
       34, 43, 40, 39, 37, 36, 38], dtype=int64)

Formatting data for Datasets

In [45]:
interactions_with_recipe_info['user_id'] = interactions_with_recipe_info.user_id.astype("str")
interactions_with_recipe_info['recipe_id'] = interactions_with_recipe_info.recipe_id.astype("str")
interactions_with_recipe_info['rating'] = interactions_with_recipe_info.rating.astype(np.float32)
interactions_with_recipe_info['recipe_name'] = interactions_with_recipe_info.recipe_name.astype("str")
interactions_with_recipe_info['tags'] = interactions_with_recipe_info.tags.astype("str")
interactions_with_recipe_info['ingredients'] = interactions_with_recipe_info.ingredients.astype("str")


Creating train preferences for users

In [46]:
user_preferences = interactions_with_recipe_info[['user_id', 'rating', 'tags_str']]
user_preferences.head()

,user_id,rating,tags_str
0,38094,4.0,"[weeknight, time-to-make, course, main-ingredi..."
1,1293707,5.0,"[weeknight, time-to-make, course, main-ingredi..."
2,8937,4.0,"[30-minutes-or-less, time-to-make, course, mai..."
3,126440,5.0,"[15-minutes-or-less, time-to-make, course, mai..."
4,57222,5.0,"[15-minutes-or-less, time-to-make, course, mai..."


In [47]:
user_liked_tags = user_preferences[user_preferences['rating'] >= 3].groupby('user_id')['tags_str'].apply(list).reset_index()
user_unliked_tags = user_preferences[user_preferences['rating'] < 3].groupby('user_id')['tags_str'].apply(list).reset_index()
users_ds = pd.DataFrame(interactions_with_recipe_info['user_id'].unique(), columns=['user_id'])


In [48]:
users_ds = users_ds.merge(user_liked_tags, on='user_id', how='left')
users_ds = users_ds.rename(columns={'tags_str': 'liked_tags'})
users_ds = users_ds.merge(user_unliked_tags, on='user_id', how='left')
users_ds = users_ds.rename(columns={'tags_str': 'unliked_tags'})

In [ ]:
users_ds.head()

Looking that some of users don't have unliked_tags, so replace all Nan with empty arrays

In [49]:
def unique_tags_in_list(tags_list):
  if isinstance(tags_list, list):
    unique_tags = []
    for sublist in tags_list:
      if isinstance(sublist, list):
        for tag in sublist:
          if tag not in unique_tags:
            unique_tags.append(tag)
    return unique_tags
  else:
    return []

users_ds['liked_tags'] = users_ds['liked_tags'].apply(unique_tags_in_list)
users_ds['unliked_tags'] = users_ds['unliked_tags'].apply(unique_tags_in_list)

In [50]:
def remove_duplicate_tags(row):
  liked_tags = set(row['liked_tags']) if isinstance(row['liked_tags'], list) else set()
  unliked_tags = set(row['unliked_tags']) if isinstance(row['unliked_tags'], list) else set()
  duplicate_tags = liked_tags.intersection(unliked_tags)
  row['liked_tags'] = [tag for tag in row['liked_tags'] if tag not in duplicate_tags] if isinstance(row['liked_tags'], list) else []
  row['unliked_tags'] = [tag for tag in row['unliked_tags'] if tag not in duplicate_tags] if isinstance(row['unliked_tags'], list) else []
  return row

users_ds = users_ds.apply(remove_duplicate_tags, axis=1)


In [ ]:
users_ds.head()

In [51]:
interactions_with_recipe_info = pd.merge(interactions_with_recipe_info, users_ds, on='user_id', how='left')

Creating datasets

In [52]:
# Параметры
MAX_TAG_LENGTH = 30
PADDING_VALUE = "empty"

interactions_with_recipe_info['liked_tags'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['liked_tags']
]
interactions_with_recipe_info['unliked_tags'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['unliked_tags']
]
interactions_with_recipe_info['ingredients_str'] =  [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['ingredients_str'] ]

interactions_with_recipe_info['tags_str'] =  [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['tags_str'] ]

raw_recipes['tags_str'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in raw_recipes['tags_str'] ]

raw_recipes['ingredients_str'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in raw_recipes['ingredients_str'] ]

In [53]:
interactions_with_recipe_info['ingredients_str'].str.len().unique(), interactions_with_recipe_info['unliked_tags'].str.len().unique(), 

(array([30], dtype=int64), array([30], dtype=int64))

In [60]:
interactions_with_recipe_info['liked_tags_str'] = interactions_with_recipe_info['liked_tags'].apply(lambda x: ','.join(x))
interactions_with_recipe_info['unliked_tags_str'] = interactions_with_recipe_info['unliked_tags'].apply(lambda x: ','.join(x))
interactions_with_recipe_info['tags_str'] = interactions_with_recipe_info['tags_str'].apply(lambda x: ','.join(x))

In [55]:
def split_tags(tags_string):
    return tf.strings.split(tf.strings.strip(tags_string), ',')

In [ ]:
tf.compat.v1.enable_eager_execution()

In [61]:
def squeeze_tags(data):
    return {
        "user_id": data["user_id"],
        "recipe_id": data["recipe_id"],
        "rating": data["rating"],
        "recipe_name": data["recipe_name"],
        "tags": tf.squeeze(data["tags"], axis=0),  # Укажите ось, которую нужно удалить
        "liked_tags": tf.squeeze(data["liked_tags"], axis=0),  # Укажите ось, которую нужно удалить
        "unliked_tags": tf.squeeze(data["unliked_tags"], axis=0)  # Укажите ось, которую нужно удалить
    }

ratings = tf.data.Dataset.from_tensor_slices((tf.cast(interactions_with_recipe_info['user_id'].values, tf.string),
                                              tf.cast(interactions_with_recipe_info['recipe_id'].values, tf.string),
                                              tf.cast(interactions_with_recipe_info['rating'].values, tf.float32),
                                              tf.cast(interactions_with_recipe_info['recipe_name'].values, tf.string),
                                              tf.cast(interactions_with_recipe_info['tags_str'].values.reshape(-1,1), tf.string),
                                              tf.cast(interactions_with_recipe_info['liked_tags_str'].values.reshape(-1,1), tf.string),
                                              tf.cast(interactions_with_recipe_info['unliked_tags_str'].values.reshape(-1,1), tf.string)
                                              )).map(lambda x, x1, x2, x3, x4, x5, x6: {
                                                  "user_id": x,
                                                  "recipe_id": x1,
                                                  "rating": x2,
                                                  "recipe_name": x3,
                                                  "tags": split_tags(x4),
                                                  "liked_tags": split_tags(x5),
                                                  "unliked_tags": split_tags(x6)
                                              }).map(squeeze_tags)


In [62]:
for data in ratings.take(1).as_numpy_iterator():
    print(data)

{'user_id': b'38094', 'recipe_id': b'40893', 'rating': 4.0, 'recipe_name': b'white bean   green chile pepper soup', 'tags': array([b'weeknight', b'time-to-make', b'course', b'main-ingredient',
       b'preparation', b'occasion', b'soups-stews', b'beans',
       b'vegetables', b'easy', b'crock-pot-slow-cooker', b'dietary',
       b'equipment', b'empty', b'empty', b'empty', b'empty', b'empty',
       b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
       b'empty', b'empty', b'empty', b'empty', b'empty', b'empty'],
      dtype=object), 'liked_tags': array([b'weeknight', b'time-to-make', b'course', b'main-ingredient',
       b'preparation', b'occasion', b'soups-stews', b'beans',
       b'vegetables', b'easy', b'crock-pot-slow-cooker', b'dietary',
       b'equipment', b'60-minutes-or-less', b'main-dish', b'fruit',
       b'vegan', b'vegetarian', b'stove-top', b'black-beans',
       b'30-minutes-or-less', b'low-protein', b'salads', b'dinner-party',
       b'low-cholesterol', b'lo

In [66]:
raw_recipes['ingredients_str'] = raw_recipes['ingredients_str'].apply(lambda x: ','.join(x))
raw_recipes['tags_str'] = raw_recipes['tags_str'].apply(lambda x: ','.join(x))

In [ ]:
raw_recipes.head()

In [67]:
raw_recipes['id'] = raw_recipes.id.astype("str")
raw_recipes['name'] = raw_recipes.name.astype("str")
raw_recipes['tags'] = raw_recipes.tags.astype("str")
raw_recipes['ingredients'] = raw_recipes.ingredients.astype("str")

In [ ]:
raw_recipes['name'][:1]

In [68]:
# prompt: Создай на основе raw_recipes датасет, но учти что ingredients_str это массив

recipes = tf.data.Dataset.from_tensor_slices((tf.cast(raw_recipes['id'].values, tf.string),
                                              tf.cast(raw_recipes['name'].values, tf.string),
                                              tf.cast(raw_recipes['tags_str'].values.reshape(-1, 1), tf.string)
                                            #   tf.cast(raw_recipes['ingredients_str'].values.reshape(-1, 1), tf.string),
                                              )
                                             ).map(lambda x, x1, x2: {
                                                  "recipe_id": x,
                                                  "recipe_name": x1,
                                                  "tags": split_tags(x2),
                                                #   "ingredients": split_tags(x3)
                                              }).map(lambda data: {
        **data,
        "tags": tf.squeeze(data["tags"], axis=0)  # Удаляем дополнительные измерения
        # "ingredients": tf.squeeze(data["ingredients"], axis=0)  # Удаляем дополнительные измерения
    })

for data in recipes.take(1).as_numpy_iterator():
  print(data.tags)


{'recipe_id': b'137739', 'recipe_name': b'arriba   baked winter squash mexican style', 'tags': array([b'60-minutes-or-less', b'time-to-make', b'course',
       b'main-ingredient', b'cuisine', b'preparation', b'occasion',
       b'north-american', b'side-dishes', b'vegetables', b'mexican',
       b'easy', b'fall', b'holiday-event', b'vegetarian', b'winter',
       b'dietary', b'christmas', b'seasonal', b'squash', b'empty',
       b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
       b'empty', b'empty', b'empty'], dtype=object)}


In [73]:
sizes = []
for data in recipes.as_numpy_iterator():
  sizes.append(len(data["tags"]))
print(np.unique(sizes))  

[30]


In [74]:
recipe_names = tf.data.Dataset.from_tensor_slices((tf.cast(raw_recipes['name'].values.reshape(-1, 1), tf.string))).map(lambda x: x)

# Towers 🗼

For our towers set dimensionality of the query and candidate representations: **32**. Higher values will correspond to models that may be more accurate, but will also be slower to fit and more prone to overfitting.

In [75]:
embedding_dimension = 32

## User tower 👷

Lets start creating our towers with User towers. We will compute by:
1. User id
2. Prefers and hates

### User ID model

In [76]:
unique_user_ids = users_ds["user_id"].unique()
unique_user_ids[:10]

array(['38094', '1293707', '8937', '126440', '57222', '52282', '124416',
       '2000192946', '76535', '273745'], dtype=object)

In [77]:
user_id_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids)+1, embedding_dimension)
])

### User Prefers models

In [78]:
# prompt: модель для liked_tags которым стоит отдать предпочтение, но учти что liked_tags в датасете это массив

unique_liked_tags = set()
for tags in users_ds['liked_tags']:
  if isinstance(tags, list):
    for tag in tags:
      unique_liked_tags.add(tag)
unique_liked_tags = list(unique_liked_tags)

liked_tags_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_liked_tags, mask_token=None),
  tf.keras.layers.Embedding(len(unique_liked_tags) + 1, embedding_dimension)
])


In [79]:
unique_liked_tags[:10]

['',
 'shakes',
 'melons',
 'easter',
 'savory-sauces',
 'main-ingredient',
 'celebrity',
 'micro-melanesia',
 'sandwiches',
 'oatmeal']

In [80]:

unique_unliked_tags = set()
for tags in users_ds['unliked_tags']:
  if isinstance(tags, list):
    for tag in tags:
      unique_unliked_tags.add(tag)
unique_unliked_tags = list(unique_unliked_tags)

unliked_tags_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_unliked_tags, mask_token=None),
  tf.keras.layers.Embedding(len(unique_unliked_tags) + 1, embedding_dimension)
])
unique_unliked_tags[:10]

['',
 'shakes',
 'melons',
 'easter',
 'savory-sauces',
 'celebrity',
 'main-ingredient',
 'micro-melanesia',
 'sandwiches',
 'oatmeal']

## Recipe tower 🌭

In [81]:
unique_recipe_names = np.unique(list(raw_recipes["name"].unique()))
recipe_name_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_recipe_names, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_recipe_names) + 1, embedding_dimension)
])
unique_recipe_names[:10]

array(['0 carb   0 cal gummy worms', '0 fat chunky watermelon salsa',
       '0 point ice cream  only 1 ingredient', '0 point soup   ww',
       '0 point soup  crock pot', '007  martini', '007 cocktail',
       '1  2  3  swiss meringue buttercream', '1 00 tangy chicken recipe',
       '1 000 artichoke hearts'], dtype='<U85')

In [82]:
# prompt: напиши вызов recipe_name_model

recipe_name_model(['some recipe name'])


<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 0.04462362, -0.0411157 ,  0.01327555, -0.00280144,  0.00288799,
        -0.02189916, -0.0080206 , -0.03798301, -0.00193814, -0.01662099,
        -0.01828358, -0.00580681,  0.02175344, -0.0115008 ,  0.04752484,
        -0.00697392, -0.01448102,  0.0471747 ,  0.02936641, -0.04120439,
         0.0456428 ,  0.02193146,  0.04568547, -0.04585805, -0.00483686,
         0.03381243, -0.00441347, -0.0137526 , -0.04958592,  0.01526498,
         0.04224307,  0.04389704]], dtype=float32)>

### Recipe ingredient&tag model

In [83]:
unique_tags = set()
for tags in raw_recipes['tags'].apply(convert_to_list):
  if isinstance(tags, list):
    for tag in tags:
      unique_tags.add(tag)
unique_tags = np.unique(list(unique_tags))

recipe_tags_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_tags, mask_token=None),
  tf.keras.layers.Embedding(len(unique_tags) + 1, embedding_dimension)
])
unique_tags[:10]

array(['', '1-day-or-more', '15-minutes-or-less', '3-steps-or-less',
       '30-minutes-or-less', '4-hours-or-less', '5-ingredients-or-less',
       '60-minutes-or-less',
       'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.',
       'a1-sauce'], dtype='<U69')

## Combine models

In [39]:
class RecipeModel(tf.keras.Model):
    def __init__(self, unique_recipe_names, unique_tags, embedding_dimension):
        super().__init__()

        self.recipe_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_recipe_names, mask_token=None),
            tf.keras.layers.Embedding(len(unique_recipe_names) + 1, embedding_dimension)
        ])
    
    # Tags embeddings
        self.tags_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_tags, mask_token=None),
            tf.keras.layers.Embedding(len(unique_tags) + 1, embedding_dimension)
        ])
        
    def call(self, features):
        # Inputs should contain "recipe_name" and "tags"
        recipe_name = features["recipe_name"]
        tags = features["tags"]

        # Embed recipe name
        recipe_embeddings = self.recipe_model(recipe_name)

        tags_embeddings = self.tags_model(tags)
        if(len(recipe_embeddings.shape.as_list())>=3): 
        #   recipe_embeddings = tf.reshape(recipe_embeddings, [1,32])
           recipe_embeddings = tf.reshape(recipe_embeddings, [1,embedding_dimension])
        if(len(tags_embeddings.shape.as_list())>=3):
            tags_embeddings = tf.reshape(tags_embeddings, [1,embedding_dimension])
        #   tags_embeddings = tf.reshape(tags_embeddings, [1,32])
        recipe_embeddings = tf.concat([recipe_embeddings, tags_embeddings], axis=1)
        return recipe_embeddings

In [85]:
class UserModel(tf.keras.Model):
    def __init__(self, unique_user_ids, unique_liked_tags, unique_unliked_tags, embedding_dimension):
        super().__init__()
        
        # User ID embeddings
        self.user_id_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])
        
        # Liked tags embeddings
        self.liked_tags_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_liked_tags, mask_token=None),
            tf.keras.layers.Embedding(len(unique_liked_tags) + 1, embedding_dimension)
        ])

        # Unliked tags embeddings
        self.unliked_tags_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_unliked_tags, mask_token=None),
            tf.keras.layers.Embedding(len(unique_unliked_tags) + 1, embedding_dimension)
        ])

        # Penalty weight for unliked tags
        self.penalty_weight = -0.5

    def call(self, features):
        user_id = features["user_id"]
        liked_tags = features["liked_tags"]
        unliked_tags = features["unliked_tags"]

        # Embed user ID
        user_id_embedding = self.user_id_model(user_id)
        #user_id_embedding = tf.reduce_mean(user_id_embedding, axis=1)

        # Embed liked and unliked tags
        liked_tags_embedding = self.liked_tags_model(liked_tags)
        unliked_tags_embedding = self.unliked_tags_model(unliked_tags)

        # Apply penalty to unliked tags
        unliked_tags_embedding = unliked_tags_embedding * self.penalty_weight

        # Aggregate embeddings
        liked_tags_embedding = tf.reduce_mean(liked_tags_embedding, axis=1, keepdims=True)  # (1, 32)
        unliked_tags_embedding = tf.reduce_mean(unliked_tags_embedding, axis=1, keepdims=True)  # (1, 32)

        # Используйте weighted суммирование для учета штрафов
        user_embedding = tf.add(user_id_embedding, liked_tags_embedding)
        user_embedding = tf.add(user_embedding, unliked_tags_embedding)

        return user_embedding



In [86]:
# Извлечение данных пользователя
test_user = {}
for data in ratings.take(1).as_numpy_iterator():
    test_user = data
    
user_model = UserModel(unique_user_ids, unique_liked_tags, unique_unliked_tags, embedding_dimension)


user_model(test_user)

<tf.Tensor: shape=(30, 32), dtype=float32, numpy=
array([[ 1.36629818e-03,  3.92483734e-02,  3.60111184e-02,
         6.65247720e-03, -1.04860514e-02,  2.09468789e-02,
         3.67291607e-02,  1.68962441e-02,  9.70973354e-03,
        -1.93486288e-02,  3.61275263e-02, -1.27276154e-02,
        -4.68134843e-02, -3.55399102e-02, -5.33879139e-02,
        -3.52501944e-02, -3.32739912e-02,  2.29207752e-03,
         1.68916248e-02, -5.63968010e-02, -9.48077068e-03,
         1.84719823e-02, -8.37540254e-03, -2.91431621e-02,
        -8.92527867e-04, -1.60721615e-02,  6.27827179e-03,
        -5.54107502e-02,  2.09747739e-02,  1.08181639e-02,
        -4.69630631e-03,  1.45586086e-02],
       [ 1.11571420e-03,  3.89977880e-02,  3.57605331e-02,
         6.40189368e-03, -1.07366359e-02,  2.06962936e-02,
         3.64785753e-02,  1.66456588e-02,  9.45915002e-03,
        -1.95992142e-02,  3.58769409e-02, -1.29781989e-02,
        -4.70640697e-02, -3.57904918e-02, -5.36384992e-02,
        -3.55007797e-0

In [87]:
class RecipeAndUserModel(tfrs.Model):

    def _reduce_mean_if_needed(self, embedding):
        """Сокращает размерность эмбеддинга до среднего, если он многомерный."""
        if len(embedding.shape) >= 3:
            return tf.reduce_mean(embedding, axis=1)
        return embedding

    def __init__(self, unique_recipe_names, unique_tags, embedding_dimension):
        super().__init__()
        self.embedding_dimension = embedding_dimension
        # Модель пользователя
        self.user_model = UserModel(unique_user_ids, unique_liked_tags, unique_unliked_tags, 64)

        # Recipe embeddings
        self.recipe_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_recipe_names, mask_token=None),
            tf.keras.layers.Embedding(len(unique_recipe_names) + 1, embedding_dimension)
        ])

        # Tags embeddings
        self.tags_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_tags, mask_token=None),
            tf.keras.layers.Embedding(len(unique_tags) + 1, embedding_dimension)
        ])

        self.dense = tf.keras.layers.Dense(embedding_dimension)  # Сжимаем до размерности 32

        # Подготовка кандидатов, учитывающих и теги
        self.candidates = recipes.batch(128).map(self._get_candidate_embeddings)

        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=self.candidates
            )
        )
        
    def _get_candidate_embeddings(self, recipe_name):
        # Получаем эмбеддинги для имен и тегов
        recipe_embedding = self.recipe_model(recipe_name['recipe_name'])
        print(recipe_name['tags'])
        tags_embedding = self.tags_model(recipe_name['tags'])  # Здесь предполагается, что recipe_name может содержать теги
        recipe_embedding = self._reduce_mean_if_needed(recipe_embedding)
        tags_embedding = self._reduce_mean_if_needed(tags_embedding)
        # Объединяем эмбеддинги имен и тегов
        combined_embedding = tf.concat([recipe_embedding, tags_embedding], axis=1)
        return combined_embedding

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # User embedding
        user_embedding = self.user_model(features)

        # Recipe name embeddings
        recipe_name_embeddings = self.recipe_model(features["recipe_name"])
        recipe_name_embeddings = self._reduce_mean_if_needed(recipe_name_embeddings)
        # Tags embeddings
        tags = features["tags"]
        tags_embeddings = self.tags_model(tags)
        tags_embeddings = self._reduce_mean_if_needed(tags_embeddings)
        # # Вычисление средневзвешенного значения для тегов (можно модифицировать веса)
        # tags_embeddings_weighted_sum = tf.reduce_sum(tags_embeddings, axis=1)
        # sum_of_weights = tf.reduce_sum(tf.ones_like(tags_embeddings), axis=1)
        # tags_embeddings = tags_embeddings_weighted_sum / sum_of_weights
        
        # recipe_name_embeddings = self._reduce_mean_if_needed(recipe_name_embeddings)
        # tags_embeddings = tf.expand_dims(tags_embeddings, axis=0)
        # Конкатенация эмбеддингов рецепта и тегов
        recipe_embeddings = tf.concat([recipe_name_embeddings, tags_embeddings], axis=1)

        return user_embedding, recipe_embeddings

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings, recipe_embeddings = self(features)
        print("Computed: ", user_embeddings.shape, recipe_embeddings.shape)
        # Используем задачу факторизованного топ-K для оптимизации
        user_embeddings = self._reduce_mean_if_needed(user_embeddings)
        recipe_embeddings = self._reduce_mean_if_needed(recipe_embeddings)
        
        # user_embeddings = tf.keras.layers.Dense(64)(user_embeddings)
        # recipe_embeddings = tf.keras.layers.Dense(64)(recipe_embeddings)
        print("Computed: ", user_embeddings.shape, recipe_embeddings.shape)
        return self.task(user_embeddings, recipe_embeddings)


In [91]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(5_000)
test = shuffled.skip(80_000).take(20_000)

cached_train =  train.batch(10_000).cache()
cached_test = test.batch(4096).cache()

In [89]:
def print_shapes(features):
    print({k: v.shape for k, v in features.items()})
    return features

# Проверьте формы данных перед batch
shuffled = shuffled.map(print_shapes)

{'user_id': TensorShape([]), 'recipe_id': TensorShape([]), 'rating': TensorShape([]), 'recipe_name': TensorShape([]), 'tags': TensorShape([None]), 'liked_tags': TensorShape([None]), 'unliked_tags': TensorShape([None])}


In [92]:
model = RecipeAndUserModel(unique_recipe_names, unique_tags, embedding_dimension)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.fit(cached_train, epochs=1)

Tensor("args_2:0", shape=(None, None), dtype=string)
Tensor("args_2:0", shape=(None, None), dtype=string)
Computed:  (None, None, 64) (None, 64)
Computed:  (None, 64) (None, 64)
Computed:  (None, None, 64) (None, 64)
Computed:  (None, 64) (None, 64)
1/1 [==============================] - 138s 138s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 42585.7461 - regularization_loss: 0.0000e+00 - total_loss: 42585.7461


In [ ]:
# prompt: Напиши код для теста model для пользователя 42
model2 = RecipeAndUserModel()
model2.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
test_user = {}
for data in test.take(1).as_numpy_iterator():
  print(model2(data))

In [93]:
model.evaluate(cached_test, return_dict=True)

Computed:  (None, None, 64) (None, 64)
Computed:  (None, 64) (None, 64)
5/5 [==============================] - 95s 17s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 2.0000e-04 - factorized_top_k/top_100_categorical_accuracy: 3.5000e-04 - loss: 32580.0553 - regularization_loss: 0.0000e+00 - total_loss: 32580.0553


{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.0,
 'factorized_top_k/top_50_categorical_accuracy': 0.00019999999494757503,
 'factorized_top_k/top_100_categorical_accuracy': 0.0003499999875202775,
 'loss': 29619.15234375,
 'regularization_loss': 0,
 'total_loss': 29619.15234375}

### Making predictions

In [98]:
# Извлечение данных пользователя
test_user = {}
for data in test.take(1).as_numpy_iterator():
    test_user = data

# Создание набора данных для тестового пользователя
user_id_to_test = "42"  # Замените на нужный ID пользователя
test_user_dataset = {
    "user_id": np.array(test_user["user_id"]),
    "liked_tags": np.array(test_user["liked_tags"]),
    "unliked_tags": np.array(test_user["unliked_tags"])
}

brute_force = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
def _reduce_mean_if_needed(embedding):
    """Сокращает размерность эмбеддинга до среднего, если он многомерный."""
    if len(embedding.shape) >= 3:
        return tf.reduce_mean(embedding, axis=1)
    return embedding
dense = tf.keras.layers.Dense(96)

def _get_candidate_embeddings(recipe_name):
    # Получаем эмбеддинги для имен и тегов
    recipe_embedding =  model.recipe_model(recipe_name['recipe_name'])
    print(recipe_name['tags'])
    tags_embedding = model.tags_model(recipe_name['tags'])  # Здесь предполагается, что recipe_name может содержать теги
    recipe_embedding = _reduce_mean_if_needed(recipe_embedding)
    tags_embedding = _reduce_mean_if_needed(tags_embedding)
    # Объединяем эмбеддинги имен и тегов
    combined_embedding = tf.concat([recipe_embedding, tags_embedding], axis=1)
    return combined_embedding
    
# Создание набора данных для рецептов и тегов
def get_recipe_and_tag_embeddings(x):
    recipe_embedding = model.recipe_model(x['recipe_name'])  # (None, 32)
    tag_embedding = model.tags_model(x['tags'])  # (None, 32)
    recipe_embedding =_reduce_mean_if_needed(recipe_embedding)
    tag_embedding = _reduce_mean_if_needed(tag_embedding)
    
    combined_embeddings = tf.concat([_reduce_mean_if_needed(recipe_embedding),  _reduce_mean_if_needed(tag_embedding)], axis=1)  # (None, 64)
    
    return dense(combined_embeddings)

# Используем map для обработки всех данных сразу
recipes_embeddings = recipes.batch(100).map(_get_candidate_embeddings)


# Создание набора имен рецептов
recipes_names = recipes.batch(100).map(lambda x: x["recipe_id"])  # Убедитесь, что это (None,)

# Объединяем эмбеддинги с именами
index = brute_force.index_from_dataset(
    tf.data.Dataset.zip((recipes_names, recipes_embeddings))
)

# Получаем рекомендации
_, titles = index(test_user_dataset, k=15)

# Вывод тегов для рекомендуемых рецептов
recommended_tags = raw_recipes[raw_recipes['id'].isin(titles.numpy().flatten().astype('str'))]


Tensor("args_2:0", shape=(None, None), dtype=string)


In [99]:
recommended_tags

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,ingredients_str,tags_str
6937,apple nut muffins,400691,60,463435,2009-11-22,"['60-minutes-or-less', 'time-to-make', 'course...","[478.7, 36.0, 123.0, 23.0, 13.0, 62.0, 20.0]",22,"['for apple filling:', 'in a saucepan , melt b...",this are like little coffee cakes to-go. perfe...,"['butter', 'brown sugar', 'flour', 'ground cin...",15,"butter,brown sugar,flour,ground cinnamon,groun...","60-minutes-or-less,time-to-make,course,main-in..."
10021,asparagus with toasted pine nuts lemon vinai...,7218,11,53425,2000-03-06,"['15-minutes-or-less', 'time-to-make', 'main-i...","[189.0, 27.0, 7.0, 12.0, 7.0, 11.0, 2.0]",13,"['snap off tough ends of asparagus', 'remove s...",NaN,"['asparagus', 'pine nuts', 'olive oil', 'lemon...",9,"asparagus,pine nuts,olive oil,lemon juice,garl...","15-minutes-or-less,time-to-make,main-ingredien..."
24339,black bean tamale pie,422184,45,1413296,2010-04-29,"['weeknight', '60-minutes-or-less', 'time-to-m...","[435.7, 33.0, 23.0, 25.0, 50.0, 36.0, 12.0]",11,"['in a skillet , saute onions and green pepper...",one of my picky family's most favorite recipes...,"['vegetable oil', 'onion', 'green pepper', 'gr...",14,"vegetable oil,onion,green pepper,ground turkey...","weeknight,60-minutes-or-less,time-to-make,cour..."
37015,carrot muffins with apple,34184,33,27643,2002-07-15,"['60-minutes-or-less', 'time-to-make', 'course...","[200.9, 10.0, 69.0, 14.0, 7.0, 5.0, 10.0]",6,"['mix first 6 ingredients in lg bowl', 'in sma...","these easy muffins are fruity, moist and delic...","['raisin bran cereal', 'flour', 'sugar', 'baki...",12,"raisin bran cereal,flour,sugar,baking soda,cin...","60-minutes-or-less,time-to-make,course,main-in..."
47671,chili mashed potatoes,52696,25,41480,2003-01-29,"['30-minutes-or-less', 'time-to-make', 'course...","[299.6, 17.0, 8.0, 5.0, 19.0, 35.0, 13.0]",7,"['in medium saucepan , cover potatoes with wat...",pep up your mashed potatoes with chili powder ...,"['potatoes', 'sour cream', 'green onion', 'chi...",9,"potatoes,sour cream,green onion,chili powder,t...","30-minutes-or-less,time-to-make,course,main-in..."
77152,easy microwave chocolate fudge,174932,5,194696,2006-06-27,"['15-minutes-or-less', 'time-to-make', 'course...","[161.0, 14.0, 60.0, 1.0, 5.0, 20.0, 5.0]",5,"['place chocolate chips , condensed milk and b...",this chocolate fudge is so easy and so yummy. ...,"['milk chocolate chips', 'sweetened condensed ...",4,"milk chocolate chips,sweetened condensed milk,...","15-minutes-or-less,time-to-make,course,main-in..."
96551,greek mama,171496,5,58104,2006-06-05,"['15-minutes-or-less', 'time-to-make', 'course...","[65.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",1,['shake with ice and strain pouring into a mar...,chill your martini glasses!!!!!,"['frangelico', 'ouzo', 'dark rum']",3,"frangelico,ouzo,dark rum,empty,empty,empty,emp...","15-minutes-or-less,time-to-make,course,cuisine..."
113248,italian style ground beef,109487,25,190300,2005-01-25,"['30-minutes-or-less', 'time-to-make', 'main-i...","[1112.8, 125.0, 5.0, 12.0, 169.0, 142.0, 1.0]",4,"['heat oil in pan until hot', 'combine green p...",this recipe is used to mix with spagetti. i le...,"['ground beef', 'fresh garlic', 'onion', 'gree...",8,"ground beef,fresh garlic,onion,green pepper,or...","30-minutes-or-less,time-to-make,main-ingredien..."
113444,italian style ground beef,218857,40,330545,2007-03-26,"['60-minutes-or-less', 'time-to-make', 'course...","[388.9, 33.0, 25.0, 27.0, 67.0, 47.0, 4.0]",10,"['mix together beef , water , seasoned salt , ...",best results come from using a lean ground mea...,"['lean ground beef', 'cold water', 'seasoning ...",11,"lean ground beef,cold water,seasoning salt,bla...","60-minutes-or-less,time-to-make,course,main-in..."
114613,java chicken,39352,500,39547,2002-09-03,"['weeknight', 'time-to-make', 'course', 'main-...","[341.9, 8.0, 161.0, 16.0, 56.0, 4.0, 14.0]",14,"['heat a saut pan over medium flame 